# McCall Model

Pablo Winant

### Job-Search Model

-   When unemployed in date, a job-seeker
    -   consumes unemployment benefit $c_t = \underline{c}$
    -   receives in every date $t$ a job offer $w_t$
        -   $w_t$ is i.i.d.,
        -   takes values $w_1, w_2, w_3$ with probabilities
            $p_1, p_2, p_3$
    -   if job-seeker accepts, becomes employed at rate $w_t$ in the
        next period
    -   else he stays unemployed
-   When employed at rate $w$
    -   worker consumes salary $c_t = w$
    -   with small probability $\lambda>0$ looses his job:
        -   starts next period unemployed
    -   otherwise stays employed at same rate
-   Objective: $\max E_0 \left\{ \sum \beta^t \log(c_t) \right\}$

**What are the states, the controls, the reward of this problem ? Write
down the Bellman equation.**

**Define a named tuple for the model.**

In [2]:
1/3

0.3333333333333333

In [5]:
m = (;
    β=0.96,
    λ=0.01,
    cbar=0.8,
    wvec=[0.6, 1.0, 1.4],
    pvec=ones(3)/3

)

(β = 0.96, λ = 0.01, cbar = 0.8, wvec = [0.6, 1.0, 1.4], pvec = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333])

**Define a function
`value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, x::Vector{Bool}, p::Parameters)::Tuple{Vector, Vector}`,
which takes in value functions tomorrow and a policy vector and return
updated values for today.**

In [11]:
function value_update(V_U, V_E, x, p)

    (;β,λ,cbar,wvec,pvec) = p

    n_V_U = zeros(3)
    n_V_E = zeros(3)

    # enumerate all unemployed states
    for n=1:3
        # compute continuation value
        if x[n] # if offer is accepted
            cont = V_E[n] 
        else
            # cont = V_U' * pvec
            cont = sum(V_U[m]*pvec[m] for m=1:3)
            # cont = sum(zip(V_U, pvec))
        end

        n_V_U[n] = log(cbar) + β*cont
    end

    # enumerate all employed states
    for n=1:3

        # receive wage
        w = wvec[n]

        # compute continuation value

        cont_U = sum(V_U[m]*pvec[m] for m=1:3) # as before
        cont_E = V_E[n]

        n_V_E[n] = log(w) + β*(λ*cont_U + (1-λ)*cont_E)
    end


    return n_V_U, n_V_E

end

value_update (generic function with 1 method)

In [12]:
V_U_0 = [0.3,2.4, 2.0]
V_E_0 = [0.3, - 476, 2.0]
x_0 = [true, false, true]
value_update(V_U_0, V_E_0, x_0, m)

([0.06485644868579027, 1.28085644868579, 1.6968564486857902], [-0.21066562376599074, -452.37536, 2.2523122366212127])

**Define a function
`policy_eval(x::Vector{Bool}, p::Parameter)::Tuple{Vector, Vector}`
which takes in a policy vector and returns the value(s) of following
this policies forever. You can add relevant arguments to the function.**

In [26]:
@time distance( (V_E_0, V_U_0), (V_E_0, V_U_0) )

  0.000006 seconds (3 allocations: 80 bytes)

0.0

In [24]:
distance(a::Tuple{Vector, Vector}, b::Tuple{Vector, Vector})  = max(
    maximum(abs, ((e-f) for (e,f) in  zip(a[1],b[1]))  ),
    maximum(abs, ((e-f) for (e,f) in  zip(a[2],b[2]))  ),
)

distance (generic function with 1 method)

In [20]:
distance( 3,2)

In [21]:
distance(a::Tuple{Vector, Vector}, b::Tuple{Vector, Vector})  = max(
    maximum(abs.(a[1]-b[1])),
    maximum(abs.(a[2]-b[2])),
)

distance (generic function with 1 method)

In [44]:
fun(a, p=.04) = a^2+p

methods(fun)

# 2 methods for generic function fun from Main: fun(a, p) in Main at /home/pablo/Teaching/ensae/mie37/tutorials/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X36sZmlsZQ==.jl:1 fun(a) in Main at /home/pablo/Teaching/ensae/mie37/tutorials/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X36sZmlsZQ==.jl:1

In [ ]:
function policy_eval(x, p; T=1000, τ_η=1e-8, verbose=true)

    V_U_0 = zeros(3)
    V_E_0 = zeros(3)
    # V_U = zeros(3)
    # V_E = zeros(3)

    local V_U, V_E

    for t=1:T

        V_U, V_E = value_update(V_U_0, V_E_0, x, p)
        
        η = distance( (V_U_0, V_E_0), (V_U, V_E))

        V_U_0, V_E_0 = V_U, V_E

        if η<τ_η
            break
        end

        if verbose
            @show (t, η)
        end

        # println("Iteration: $n ; η = $η")

    end

    return (; V_U, V_E)
    
end

policy_eval (generic function with 1 method)

In [38]:
policy_eval([false, false, false], m; T=200, verbose=false)

(V_U = [-5.577001073670461, -5.577001073670461, -5.577001073670461], V_E = [-11.376821691845214, -1.0783107010132913, 5.705145072812092])

In [41]:
policy_eval([true, true, true], m;)

(t, η) = (1, 0.5108256237659907)
(t, η) = (2, 0.490392598815351)
(t, η) = (3, 0.46812561688302146)
(t, η) = (4, 0.4454207753467201)
(t, η) = (5, 0.42383554279395796)
(t, η) = (6, 0.4033004977021406)
(t, η) = (7, 0.38376450422539854)
(t, η) = (8, 0.36517878756433)
(t, η) = (9, 0.34749696233982963)
(t, η) = (10, 0.3306749139409164)
(t, η) = (11, 0.31467068710526247)
(t, η) = (12, 0.2994443799648252)
(t, η) = (13, 0.28495804330095265)
(t, η) = (14, 0.2711755847523154)
(t, η) = (15, 0.25806267773172475)
(t, η) = (16, 0.24558667482003127)
(t, η) = (17, 0.23371652541660648)
(t, η) = (18, 0.22242269743688414)
(t, η) = (19, 0.21167710285771957)
(t, η) = (20, 0.20145302692113276)
(t, η) = (21, 0.1917250608163963)
(t, η) = (22, 0.18246903766922973)
(t, η) = (23, 0.17366197167539088)
(t, η) = (24, 0.16528200022386397)
(t, η) = (25, 0.15730832886259094)
(t, η) = (26, 0.14972117896684178)
(t, η) = (27, 0.14250173797729015)
(t, η) = (28, 0.13563211208133197)
(t, η) = (29, 0.1290952812175039)
(t, η) 

(V_U = [-10.417717805631884, -0.5307702781758938, 5.9815955726093115], V_E = [-10.619348191494039, -0.320444515546618, 6.463269913351949])

In [42]:
policy_eval(x_0, m;)

(t, η) = (1, 0.5108256237659907)
(t, η) = (2, 0.490392598815351)
(t, η) = (3, 0.46812561688302146)
(t, η) = (4, 0.4460788524567716)
(t, η) = (5, 0.4248297869138624)
(t, η) = (6, 0.40452234933286846)
(t, η) = (7, 0.3851676119158367)
(t, η) = (8, 0.3667370420134395)
(t, η) = (9, 0.3491913678231824)
(t, η) = (10, 0.332489462473883)
(t, η) = (11, 0.31659102952791507)
(t, η) = (12, 0.30145736797222167)
(t, η) = (13, 0.2870515444381647)
(t, η) = (14, 0.27333838461626314)
(t, η) = (15, 0.2602844115834575)
(t, η) = (16, 0.24785777053778268)
(t, η) = (17, 0.2360281520580898)
(t, η) = (18, 0.22476671751561916)
(t, η) = (19, 0.2140460276352858)
(t, η) = (20, 0.20383997439472346)
(t, η) = (21, 0.19412371620383695)
(t, η) = (22, 0.18487361623708498)
(t, η) = (23, 0.17606718377422315)
(t, η) = (24, 0.16768301840536726)
(t, η) = (25, 0.1597007569609339)
(t, η) = (26, 0.152101023033401)
(t, η) = (27, 0.14486537896396268)
(t, η) = (28, 0.1379762801734481)
(t, η) = (29, 0.13141703172269814)
(t, η) = (30

(V_U = [-10.56046812449, -2.5500924348984393, 5.838845271607704], V_E = [-10.768046440045865, -0.46914275344062545, 6.314571682478065])

**Define a function
`bellman_step(V_E::Vector, V_U::Vector, p::Parameters)::Tuple{Vector, Vector, Vector}`
which returns updated values, together with improved policy rules.**

In [59]:
function bellman_step(V_U, V_E, p)

    (;β,λ,cbar,wvec,pvec) = p


    n_x = zeros(Bool, 3)

    n_V_U = zeros(3)
    n_V_E = zeros(3)

    # enumerate all unemployed states
    for n=1:3
        # compute continuation value

        # if accept
        cont_accept = V_E[n] 
        
        # if reject
        cont_reject = sum(V_U[m]*pvec[m] for m=1:3)

        if cont_accept > cont_reject
            n_x[n] = true
            n_V_U[n] = log(cbar) + β*cont_accept
        else
            n_x[n] = false
            n_V_U[n] = log(cbar) + β*cont_reject

        end

    end

    # enumerate all employed states
    for n=1:3

        # receive wage
        w = wvec[n]

        # compute continuation value

        cont_U = sum(V_U[m]*pvec[m] for m=1:3) # as before
        cont_E = V_E[n]

        n_V_E[n] = log(w) + β*(λ*cont_U + (1-λ)*cont_E)
    end
    return (;n_V_U, n_V_E, n_x)
end

bellman_step (generic function with 1 method)

In [ ]:
bellman_step(V_U_0, V_E_0, m)

(n_V_U = [0.06485644868579027, 2.08085644868579, 1.6968564486857902], n_V_E = [-1.741545623765991, 0.7651199999999997, 0.7214322366212127], n_x = Bool[1, 1, 1])

**Implement Value Function Iteration**

In [84]:
function value_iteration(p; T=1000, τ_η=1e-8, verbose=true)

    V_U_0 = zeros(3)
    V_E_0 = zeros(3)

    local V_U, V_E

    for t=1:T

        V_U, V_E, x = bellman_step(V_U_0, V_E_0, p)
        
        η = distance( (V_U_0, V_E_0), (V_U, V_E))

        V_U_0, V_E_0 = V_U, V_E

        if η<τ_η
            break
        end

        if verbose
            @show (t, η, x)
        end

        # println("Iteration: $n ; η = $η")

    end

    return (; V_U, V_E)
    
end

value_iteration (generic function with 1 method)

In [58]:
m

(β = 0.96, λ = 0.01, cbar = 0.8, wvec = [0.6, 1.0, 1.4], pvec = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333])

In [62]:
value_iteration(m)

(t, η, x) = (1, 0.5108256237659907, Bool[0, 0, 0])
(t, η, x) = (2, 0.48763085091981395, Bool[0, 1, 1])
(t, η, x) = (3, 0.4630962149929281, Bool[0, 1, 1])
(t, η, x) = (4, 0.43904602360823586, Bool[0, 1, 1])
(t, η, x) = (5, 0.4160002641604996, Bool[0, 0, 1])
(t, η, x) = (6, 0.39388523729722946, Bool[0, 0, 1])
(t, η, x) = (7, 0.3725547270934362, Bool[0, 0, 1])
(t, η, x) = (8, 0.3521196771442874, Bool[0, 0, 1])
(t, η, x) = (9, 0.3326287657342153, Bool[0, 0, 1])
(t, η, x) = (10, 0.31409252865426396, Bool[0, 0, 1])
(t, η, x) = (11, 0.2964985089906609, Bool[0, 0, 1])
(t, η, x) = (12, 0.2798207532429604, Bool[0, 0, 1])
(t, η, x) = (13, 0.26402574614661223, Bool[0, 0, 1])
(t, η, x) = (14, 0.24907610309183692, Bool[0, 0, 1])
(t, η, x) = (15, 0.2349328514342366, Bool[0, 0, 1])
(t, η, x) = (16, 0.2215568246184949, Bool[0, 0, 1])
(t, η, x) = (17, 0.20890949927437052, Bool[0, 0, 1])
(t, η, x) = (18, 0.1969534833026243, Bool[0, 0, 1])
(t, η, x) = (19, 0.18565278597628865, Bool[0, 0, 1])
(t, η, x) = (

(V_U = [6.049790992670206, 6.049790992670206, 7.503338493801174], V_E = [-9.034199339279674, 1.2647043614819962, 8.048418806725287])

**Implement Policy Iteration and compare rates of convergence.**

In [64]:
[true, false, true] == [true, true, true]

false

In [74]:
function policy_iteration(p; T=1000,  verbose=true)


    local V_U, V_E, x
    
    x_0 = zeros(Bool, 3)
    V_U_0, V_E_0 = policy_eval(x_0, p; verbose=false)

    for t=1:T

        V_U_t, V_E_t, x = bellman_step(V_U_0, V_E_0, p)

        V_U, V_E = policy_eval(x, p; verbose=false)
        
        V_U_0, V_E_0 = V_U, V_E

        if verbose
            @show (t, x, V_U, V_E)
        end

        if (x==x_0)
            break
        end

        x_0 = x


    end

    return (; V_U, V_E, x)
    
end

policy_iteration (generic function with 1 method)

In [75]:
policy_iteration(m)

(t, x, V_U, V_E) = (1, Bool[0, 1, 1], [3.1389163916366356, 0.42757754738899856, 6.939943411173783], [-9.621069208340083, 0.6778344871456846, 7.461548928913835])
(t, x, V_U, V_E) = (2, Bool[0, 0, 1], [6.049790992524931, 6.049790992524931, 7.503338493655899], [-9.034199339424942, 1.264704361336724, 8.048418806580012])
(t, x, V_U, V_E) = (3, Bool[0, 0, 1], [6.049790992524931, 6.049790992524931, 7.503338493655899], [-9.034199339424942, 1.264704361336724, 8.048418806580012])

(V_U = [6.049790992524931, 6.049790992524931, 7.503338493655899], V_E = [-9.034199339424942, 1.264704361336724, 8.048418806580012], x = Bool[0, 0, 1])

**Discuss the Effects of the Parameters**

In [78]:
m

(β = 0.96, λ = 0.01, cbar = 0.8, wvec = [0.6, 1.0, 1.4], pvec = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333])

In [ ]:
policy_iteration(m; verbose=false).x

3-element Vector{Bool}:
 0
 0
 1

In [81]:
merge(m, (;cbar=0.5))

(β = 0.96, λ = 0.01, cbar = 0.5, wvec = [0.6, 1.0, 1.4], pvec = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333])

In [97]:
policy_iteration(merge(m, (;cbar=0.5, β=0.1)); verbose=false).x

3-element Vector{Bool}:
 1
 1
 1

In [ ]:
policy_iteration(merge(m, (;cbar=0.5, λ=0.2)); verbose=false).x

3-element Vector{Bool}:
 0
 1
 1

In [ ]:
policy_iteration(merge(m, (;cbar=0.6, wvec=[0.9, 1.0, 1.1], λ=0.1)); verbose=false).xa

3-element Vector{Bool}:
 0
 1
 1

In [83]:
value_iteration(merge(m, (;cbar=0.5)))

(t, η, x) = (1, 0.6931471805599453, Bool[0, 0, 0])
(t, η, x) = (2, 0.4921428857605732, Bool[1, 1, 1])
(t, η, x) = (3, 0.4682682122321573, Bool[0, 1, 1])
(t, η, x) = (4, 0.4448321065334351, Bool[0, 1, 1])
(t, η, x) = (5, 0.421809731483338, Bool[0, 1, 1])
(t, η, x) = (6, 0.3997326104086891, Bool[0, 1, 1])
(t, η, x) = (7, 0.37872378365529347, Bool[0, 1, 1])
(t, η, x) = (8, 0.3587816360837337, Bool[0, 1, 1])
(t, η, x) = (9, 0.3398678888101081, Bool[0, 1, 1])
(t, η, x) = (10, 0.3219348089922658, Bool[0, 1, 1])
(t, η, x) = (11, 0.3049335580865655, Bool[0, 0, 1])
(t, η, x) = (12, 0.288499138959482, Bool[0, 0, 1])
(t, η, x) = (13, 0.27272309403811246, Bool[0, 0, 1])
(t, η, x) = (14, 0.2576563449970104, Bool[0, 0, 1])
(t, η, x) = (15, 0.24331537237385614, Bool[0, 0, 1])
(t, η, x) = (16, 0.22969576080372534, Bool[0, 0, 1])
(t, η, x) = (17, 0.2167806958310372, Bool[0, 0, 1])
(t, η, x) = (18, 0.20454628035002465, Bool[0, 0, 1])
(t, η, x) = (19, 0.192964848183526, Bool[0, 0, 1])
(t, η, x) = (20, 0.

(V_U = [4.066009442589008, 4.066009442589008, 6.740345591111497], V_E = [-9.339396498834146, 0.9595072017084403, 7.743221646807431])